In [1]:
import pandas as pd
from io import StringIO
import json
from pyproj import Proj
import numpy as np

In [2]:
picks = """station_id,phase_index,phase_time,phase_score,phase_type,phase_polarity,phase_amplitude,id,timestamp,amp,type,prob,event_index,gamma_score
CI.CLC..BH,6021,2019-07-04T21:01:00.204,0.656,P,0.366,8.148e-05,CI.CLC..BH,2019-07-04T21:01:00.204000,8.148e-05,P,0.656,33,7.69045483708768e-05
CI.CLC..HH,6021,2019-07-04T21:01:00.208,0.953,P,0.867,9.035e-05,CI.CLC..HH,2019-07-04T21:01:00.208000,9.035e-05,P,0.953,33,6.921450648757625e-05
CI.CLC..HN,6021,2019-07-04T21:01:00.208,0.946,P,0.883,9.184e-05,CI.CLC..HN,2019-07-04T21:01:00.208000,9.184e-05,P,0.946,33,0.10224616208562531
CI.SRT..BH,6038,2019-07-04T21:01:00.374,0.695,P,0.282,1.334e-05,CI.SRT..BH,2019-07-04T21:01:00.374000,1.334e-05,P,0.695,33,0.13650090668380369
CI.SRT..HH,6048,2019-07-04T21:01:00.478,0.887,P,0.773,1.117e-05,CI.SRT..HH,2019-07-04T21:01:00.478000,1.117e-05,P,0.887,33,0.13338778112653066
CI.SRT..HN,6048,2019-07-04T21:01:00.478,0.822,P,0.587,9.291e-06,CI.SRT..HN,2019-07-04T21:01:00.478000,9.291e-06,P,0.822,33,0.10582383432535326
CI.LRL..BH,6140,2019-07-04T21:01:01.394,0.642,P,-0.369,2.252e-05,CI.LRL..BH,2019-07-04T21:01:01.394000,2.252e-05,P,0.642,33,0.10923449330211399
CI.LRL..HH,6145,2019-07-04T21:01:01.448,0.839,P,-0.659,2.324e-05,CI.LRL..HH,2019-07-04T21:01:01.448000,2.324e-05,P,0.839,33,0.07759740585152225
CI.LRL..HN,6145,2019-07-04T21:01:01.448,0.797,P,-0.542,2.574e-05,CI.LRL..HN,2019-07-04T21:01:01.448000,2.574e-05,P,0.797,33,0.1335516794747089
CI.TOW2..BH,6153,2019-07-04T21:01:01.524,0.881,P,0.927,4.392e-05,CI.TOW2..BH,2019-07-04T21:01:01.524000,4.392e-05,P,0.881,33,0.19283358210022641
CI.TOW2..HH,6155,2019-07-04T21:01:01.548,0.851,P,0.926,1.427e-05,CI.TOW2..HH,2019-07-04T21:01:01.548000,1.427e-05,P,0.851,33,0.18177901272816355
CI.TOW2..HN,6155,2019-07-04T21:01:01.548,0.776,P,0.838,3.504e-05,CI.TOW2..HN,2019-07-04T21:01:01.548000,3.504e-05,P,0.776,33,0.2011528242151
CI.CCC..BH,6231,2019-07-04T21:01:02.304,0.842,P,0.823,3.601e-05,CI.CCC..BH,2019-07-04T21:01:02.304000,3.601e-05,P,0.842,33,0.15661319563879
CI.CCC..HH,6235,2019-07-04T21:01:02.348,0.939,P,0.899,4.489e-05,CI.CCC..HH,2019-07-04T21:01:02.348000,4.489e-05,P,0.939,33,0.18422330810225462
CI.CCC..HN,6235,2019-07-04T21:01:02.348,0.927,P,0.859,3.2e-05,CI.CCC..HN,2019-07-04T21:01:02.348000,3.2e-05,P,0.927,33,0.17266930089157423
CI.WRC2..BH,6260,2019-07-04T21:01:02.594,0.884,P,0.815,4.955e-05,CI.WRC2..BH,2019-07-04T21:01:02.594000,4.955e-05,P,0.884,33,0.15646073136412014
CI.CLC..BH,6262,2019-07-04T21:01:02.614,0.778,S,-0.101,8.148e-05,CI.CLC..BH,2019-07-04T21:01:02.614000,8.148e-05,S,0.778,33,0.18498183923718553
CI.CLC..HH,6262,2019-07-04T21:01:02.618,0.883,S,-0.129,9.035e-05,CI.CLC..HH,2019-07-04T21:01:02.618000,9.035e-05,S,0.883,33,0.1506314881118966
CI.CLC..HN,6262,2019-07-04T21:01:02.618,0.908,S,-0.238,9.184e-05,CI.CLC..HN,2019-07-04T21:01:02.618000,9.184e-05,S,0.908,33,0.051229030895310036
CI.WBM..BH,6263,2019-07-04T21:01:02.626,0.932,P,-0.896,2.577e-05,CI.WBM..BH,2019-07-04T21:01:02.626000,2.577e-05,P,0.932,33,0.053491628681719444
CI.WRC2..HH,6263,2019-07-04T21:01:02.628,0.912,P,0.87,5.09e-05,CI.WRC2..HH,2019-07-04T21:01:02.628000,5.09e-05,P,0.912,33,0.054096419158907685
CI.WBM..HH,6265,2019-07-04T21:01:02.653,0.939,P,-0.902,2.69e-05,CI.WBM..HH,2019-07-04T21:01:02.653000,2.69e-05,P,0.939,33,0.20517999682833393
CI.WBM..HN,6265,2019-07-04T21:01:02.653,0.872,P,-0.826,2.753e-05,CI.WBM..HN,2019-07-04T21:01:02.653000,2.753e-05,P,0.872,33,0.15086660478069383
CI.WRC2..HN,6266,2019-07-04T21:01:02.658,0.9,P,0.88,4.195e-05,CI.WRC2..HN,2019-07-04T21:01:02.658000,4.195e-05,P,0.9,33,0.19487292963544423
CI.WNM..EH,6323,2019-07-04T21:01:03.230,0.912,P,0.397,2.18e-06,CI.WNM..EH,2019-07-04T21:01:03.230000,2.18e-06,P,0.912,33,0.17976412745500267
CI.WNM..HN,6326,2019-07-04T21:01:03.260,0.831,P,0.296,1.653e-06,CI.WNM..HN,2019-07-04T21:01:03.260000,1.653e-06,P,0.831,33,0.1783978872744624
CI.WVP2..EH,6351,2019-07-04T21:01:03.510,0.84,P,0.88,3.9e-06,CI.WVP2..EH,2019-07-04T21:01:03.510000,3.9e-06,P,0.84,33,0.031928679597777586
CI.SRT..BH,6352,2019-07-04T21:01:03.514,0.765,S,0.0,6.608e-05,CI.SRT..BH,2019-07-04T21:01:03.514000,6.608e-05,S,0.765,33,0.015090497781846988
CI.SRT..HH,6352,2019-07-04T21:01:03.518,0.759,S,0.0,6.865e-05,CI.SRT..HH,2019-07-04T21:01:03.518000,6.865e-05,S,0.759,33,0.10224681619833037
CI.SRT..HN,6353,2019-07-04T21:01:03.528,0.736,S,0.0,5.686e-05,CI.SRT..HN,2019-07-04T21:01:03.528000,5.686e-05,S,0.736,33,0.13407167507500936
CI.SLA..BH,6355,2019-07-04T21:01:03.544,0.886,P,-0.817,1.376e-05,CI.SLA..BH,2019-07-04T21:01:03.544000,1.376e-05,P,0.886,33,0.12944568485642424
CI.SLA..HH,6355,2019-07-04T21:01:03.548,0.91,P,-0.869,1.493e-05,CI.SLA..HH,2019-07-04T21:01:03.548000,1.493e-05,P,0.91,33,0.13263801088374963
CI.SLA..HN,6357,2019-07-04T21:01:03.568,0.89,P,-0.806,1.388e-05,CI.SLA..HN,2019-07-04T21:01:03.568000,1.388e-05,P,0.89,33,0.2512461711771998
CI.WVP2..HN,6366,2019-07-04T21:01:03.660,0.814,P,-0.655,8.173e-06,CI.WVP2..HN,2019-07-04T21:01:03.660000,8.173e-06,P,0.814,33,0.25777057220091437
CI.JRC2..BH,6397,2019-07-04T21:01:03.964,0.933,P,0.871,1.201e-05,CI.JRC2..BH,2019-07-04T21:01:03.964000,1.201e-05,P,0.933,33,0.24923969152725187
CI.JRC2..HN,6402,2019-07-04T21:01:04.018,0.804,P,0.609,1.16e-05,CI.JRC2..HN,2019-07-04T21:01:04.018000,1.16e-05,P,0.804,33,0.18989687926758878
CI.JRC2..HH,6403,2019-07-04T21:01:04.028,0.956,P,0.9,1.306e-05,CI.JRC2..HH,2019-07-04T21:01:04.028000,1.306e-05,P,0.956,33,0.26857290464282735
CI.WCS2..BH,6432,2019-07-04T21:01:04.314,0.754,P,-0.28,1.192e-05,CI.WCS2..BH,2019-07-04T21:01:04.314000,1.192e-05,P,0.754,33,0.22696978788999983
CI.WCS2..HH,6432,2019-07-04T21:01:04.318,0.792,P,-0.48,1.32e-05,CI.WCS2..HH,2019-07-04T21:01:04.318000,1.32e-05,P,0.792,33,0.2700677937640668
CI.WCS2..HN,6443,2019-07-04T21:01:04.428,0.682,P,0.66,1.134e-05,CI.WCS2..HN,2019-07-04T21:01:04.428000,1.134e-05,P,0.682,33,0.21757948906397948
CI.LRL..BH,6474,2019-07-04T21:01:04.734,0.812,S,0.0,3.508e-05,CI.LRL..BH,2019-07-04T21:01:04.734000,3.508e-05,S,0.812,33,0.22675557594185433
CI.LRL..HH,6474,2019-07-04T21:01:04.738,0.777,S,0.0,4.095e-05,CI.LRL..HH,2019-07-04T21:01:04.738000,4.095e-05,S,0.777,33,0.18737379853160702
CI.LRL..HN,6477,2019-07-04T21:01:04.768,0.759,S,0.0,3.871e-05,CI.LRL..HN,2019-07-04T21:01:04.768000,3.871e-05,S,0.759,33,0.06812199175735986
CI.WRV2..EH,6497,2019-07-04T21:01:04.970,0.92,P,0.892,1.549e-06,CI.WRV2..EH,2019-07-04T21:01:04.970000,1.549e-06,P,0.92,33,0.06025537881362931
CI.WRV2..HN,6502,2019-07-04T21:01:05.020,0.839,P,-0.632,1.375e-06,CI.WRV2..HN,2019-07-04T21:01:05.020000,1.375e-06,P,0.839,33,0.06827232252609507
CI.TOW2..BH,6547,2019-07-04T21:01:05.464,0.755,S,0.0,4.392e-05,CI.TOW2..BH,2019-07-04T21:01:05.464000,4.392e-05,S,0.755,33,0.03938825688094089
CI.TOW2..HH,6547,2019-07-04T21:01:05.468,0.739,S,0.0,5.152e-05,CI.TOW2..HH,2019-07-04T21:01:05.468000,5.152e-05,S,0.739,33,0.02688985996995693
CI.TOW2..HN,6553,2019-07-04T21:01:05.528,0.695,S,0.0,3.504e-05,CI.TOW2..HN,2019-07-04T21:01:05.528000,3.504e-05,S,0.695,33,0.02195476810594709
CI.DTP..HH,6577,2019-07-04T21:01:05.768,0.909,P,-0.866,9.358e-07,CI.DTP..HH,2019-07-04T21:01:05.768000,9.358e-07,P,0.909,33,0.019765792869094863
CI.DTP..BH,6579,2019-07-04T21:01:05.784,0.932,P,-0.892,9.007e-07,CI.DTP..BH,2019-07-04T21:01:05.784000,9.007e-07,P,0.932,33,0.014855552664743423
CI.DTP..HN,6583,2019-07-04T21:01:05.828,0.766,P,-0.397,3.443e-06,CI.DTP..HN,2019-07-04T21:01:05.828000,3.443e-06,P,0.766,33,0.020373459466434228
CI.CCC..BH,6621,2019-07-04T21:01:06.204,0.76,S,-0.001,3.601e-05,CI.CCC..BH,2019-07-04T21:01:06.204000,3.601e-05,S,0.76,33,0.01917606576145559
CI.CCC..HH,6622,2019-07-04T21:01:06.218,0.757,S,0.0,4.489e-05,CI.CCC..HH,2019-07-04T21:01:06.218000,4.489e-05,S,0.757,33,0.14456339491438686
CI.CCC..HN,6625,2019-07-04T21:01:06.248,0.707,S,0.0,3.2e-05,CI.CCC..HN,2019-07-04T21:01:06.248000,3.2e-05,S,0.707,33,0.1417942388021207
CI.WMF..HN,6648,2019-07-04T21:01:06.478,0.723,P,0.279,1.847e-06,CI.WMF..HN,2019-07-04T21:01:06.478000,1.847e-06,P,0.723,33,0.12061130536269296
CI.WMF..BH,6651,2019-07-04T21:01:06.504,0.926,P,0.898,1.067e-06,CI.WMF..BH,2019-07-04T21:01:06.504000,1.067e-06,P,0.926,33,0.12631419873592697
CI.WMF..HH,6652,2019-07-04T21:01:06.518,0.937,P,0.901,1.16e-06,CI.WMF..HH,2019-07-04T21:01:06.518000,1.16e-06,P,0.937,33,0.062103324599718374
CI.WRC2..BH,6671,2019-07-04T21:01:06.704,0.682,S,0.0,4.955e-05,CI.WRC2..BH,2019-07-04T21:01:06.704000,4.955e-05,S,0.682,33,0.027785364929419798
CI.WRC2..HH,6673,2019-07-04T21:01:06.728,0.713,S,0.0,5.09e-05,CI.WRC2..HH,2019-07-04T21:01:06.728000,5.09e-05,S,0.713,33,0.03279866232697586
CI.WRC2..HN,6674,2019-07-04T21:01:06.738,0.656,S,0.0,4.195e-05,CI.WRC2..HN,2019-07-04T21:01:06.738000,4.195e-05,S,0.656,33,0.10783545886041383
CI.WBM..HH,6676,2019-07-04T21:01:06.763,0.803,S,0.0,2.69e-05,CI.WBM..HH,2019-07-04T21:01:06.763000,2.69e-05,S,0.803,33,0.11184171500944862
CI.WBM..BH,6677,2019-07-04T21:01:06.766,0.793,S,-0.005,2.577e-05,CI.WBM..BH,2019-07-04T21:01:06.766000,2.577e-05,S,0.793,33,0.12598264125192246
CI.WBM..HN,6678,2019-07-04T21:01:06.783,0.721,S,0.0,2.753e-05,CI.WBM..HN,2019-07-04T21:01:06.783000,2.753e-05,S,0.721,33,0.15769581870468335
CI.WNM..EH,6779,2019-07-04T21:01:07.790,0.508,S,0.0,7.991e-06,CI.WNM..EH,2019-07-04T21:01:07.790000,7.991e-06,S,0.508,33,0.15642768672645377
CI.WNM..HN,6781,2019-07-04T21:01:07.810,0.778,S,0.0,9.467e-06,CI.WNM..HN,2019-07-04T21:01:07.810000,9.467e-06,S,0.778,33,0.13489939792952044
CI.WVP2..HN,6829,2019-07-04T21:01:08.290,0.667,S,0.0,8.173e-06,CI.WVP2..HN,2019-07-04T21:01:08.290000,8.173e-06,S,0.667,33,0.11608419501690612
CI.SLA..HH,6848,2019-07-04T21:01:08.478,0.813,S,0.0,3.62e-05,CI.SLA..HH,2019-07-04T21:01:08.478000,3.62e-05,S,0.813,33,0.19726807570338137
CI.SLA..BH,6849,2019-07-04T21:01:08.484,0.825,S,0.0,3.745e-05,CI.SLA..BH,2019-07-04T21:01:08.484000,3.745e-05,S,0.825,33,0.1441600752141464
CI.SLA..HN,6849,2019-07-04T21:01:08.488,0.783,S,0.0,2.888e-05,CI.SLA..HN,2019-07-04T21:01:08.488000,2.888e-05,S,0.783,33,0.14720747050678032
CI.WVP2..EH,6856,2019-07-04T21:01:08.560,0.384,S,0.007,3.9e-06,CI.WVP2..EH,2019-07-04T21:01:08.560000,3.9e-06,S,0.384,33,0.1441492922175336
CI.JRC2..BH,6906,2019-07-04T21:01:09.054,0.792,S,0.0,1.201e-05,CI.JRC2..BH,2019-07-04T21:01:09.054000,1.201e-05,S,0.792,33,0.17123878376480856
CI.JRC2..HH,6906,2019-07-04T21:01:09.058,0.796,S,0.0,1.306e-05,CI.JRC2..HH,2019-07-04T21:01:09.058000,1.306e-05,S,0.796,33,0.04570923602666838
CI.JRC2..HN,6908,2019-07-04T21:01:09.078,0.823,S,0.0,1.16e-05,CI.JRC2..HN,2019-07-04T21:01:09.078000,1.16e-05,S,0.823,33,0.20865366823037845
CI.WCS2..BH,6965,2019-07-04T21:01:09.644,0.755,S,-0.0,1.192e-05,CI.WCS2..BH,2019-07-04T21:01:09.644000,1.192e-05,S,0.755,33,0.21021885093961226
CI.WCS2..HH,6965,2019-07-04T21:01:09.648,0.765,S,0.0,1.32e-05,CI.WCS2..HH,2019-07-04T21:01:09.648000,1.32e-05,S,0.765,33,0.22165735137950715
CI.WCS2..HN,6971,2019-07-04T21:01:09.708,0.79,S,0.0,1.134e-05,CI.WCS2..HN,2019-07-04T21:01:09.708000,1.134e-05,S,0.79,33,0.18975550885168213
CI.WRV2..EH,7018,2019-07-04T21:01:10.180,0.313,S,-0.015,2.556e-06,CI.WRV2..EH,2019-07-04T21:01:10.180000,2.556e-06,S,0.313,33,0.19093662677022483
CI.WRV2..HN,7032,2019-07-04T21:01:10.320,0.356,S,0.0,3.034e-06,CI.WRV2..HN,2019-07-04T21:01:10.320000,3.034e-06,S,0.356,33,0.21352881367817966
CI.DTP..HH,7215,2019-07-04T21:01:12.148,0.513,S,0.0,3.303e-06,CI.DTP..HH,2019-07-04T21:01:12.148000,3.303e-06,S,0.513,33,0.006637870427029029
CI.DTP..BH,7224,2019-07-04T21:01:12.234,0.532,S,0.0,3.332e-06,CI.DTP..BH,2019-07-04T21:01:12.234000,3.332e-06,S,0.532,33,0.016464644287189925
CI.DTP..HN,7225,2019-07-04T21:01:12.248,0.611,S,0.0,3.443e-06,CI.DTP..HN,2019-07-04T21:01:12.248000,3.443e-06,S,0.611,33,0.05053245080407329
CI.WMF..BH,7327,2019-07-04T21:01:13.264,0.68,S,0.0,3.572e-06,CI.WMF..BH,2019-07-04T21:01:13.264000,3.572e-06,S,0.68,33,0.06548451596472597
CI.WMF..HH,7327,2019-07-04T21:01:13.268,0.665,S,0.0,3.845e-06,CI.WMF..HH,2019-07-04T21:01:13.268000,3.845e-06,S,0.665,33,0.0798476576010151
CI.WMF..HN,7329,2019-07-04T21:01:13.288,0.707,S,0.025,3.323e-06,CI.WMF..HN,2019-07-04T21:01:13.288000,3.323e-06,S,0.707,33,0.00233987235550235
"""
picks = pd.read_csv(StringIO(picks), parse_dates=['phase_time'])

In [3]:
stations = """
network,station,location,instrument,component,sensitivity,latitude,longitude,elevation_m,depth_km,x_km,y_km,z_km,provider,station_id
CI,CCC,,BH,ENZ,"[627368000.0, 627368000.0, 627368000.0]",35.52495,-117.36453,670.0,-0.67,12.65,-19.968,-0.67,SCEDC,CI.CCC..BH
CI,CCC,,HH,ENZ,"[627368000.0, 627368000.0, 627368000.0]",35.52495,-117.36453,670.0,-0.67,12.65,-19.968,-0.67,SCEDC,CI.CCC..HH
CI,CCC,,HN,ENZ,"[213979.0, 214322.0, 213808.0]",35.52495,-117.36453,670.0,-0.67,12.65,-19.968,-0.67,SCEDC,CI.CCC..HN
CI,CLC,,BH,ENZ,"[627373000.0, 627373000.0, 627373000.0]",35.81574,-117.59751,775.0,-0.775,-8.451,12.291,-0.775,SCEDC,CI.CLC..BH
CI,CLC,,HH,ENZ,"[627369000.0, 627369000.0, 627369000.0]",35.81574,-117.59751,775.0,-0.775,-8.451,12.291,-0.775,SCEDC,CI.CLC..HH
CI,CLC,,HN,ENZ,"[213945.0, 213809.0, 213740.0]",35.81574,-117.59751,775.0,-0.775,-8.451,12.291,-0.775,SCEDC,CI.CLC..HN
CI,DTP,,BH,ENZ,"[627368000.0, 627368000.0, 627368000.0]",35.26742,-117.84581,908.0,-0.908,-31.102,-48.496,-0.908,SCEDC,CI.DTP..BH
CI,DTP,,HH,ENZ,"[627368000.0, 627368000.0, 627368000.0]",35.26742,-117.84581,908.0,-0.908,-31.102,-48.496,-0.908,SCEDC,CI.DTP..HH
CI,DTP,,HN,ENZ,"[214399.0, 213971.0, 214484.0]",35.26742,-117.84581,908.0,-0.908,-31.102,-48.496,-0.908,SCEDC,CI.DTP..HN
CI,JRC2,,BH,ENZ,"[784866000.0, 784866000.0, 790478000.0]",35.98249,-117.80885,1469.0,-1.469,-27.493,30.832,-1.469,SCEDC,CI.JRC2..BH
CI,JRC2,,HH,ENZ,"[784866000.0, 784866000.0, 790478000.0]",35.98249,-117.80885,1469.0,-1.469,-27.493,30.832,-1.469,SCEDC,CI.JRC2..HH
CI,JRC2,,HN,ENZ,"[213808.0, 213945.0, 214185.0]",35.98249,-117.80885,1469.0,-1.469,-27.493,30.832,-1.469,SCEDC,CI.JRC2..HN
CI,LRL,,BH,ENZ,"[628306000.0, 629984000.0, 627467000.0]",35.47954,-117.68212,1340.0,-1.34,-16.165,-25.001,-1.34,SCEDC,CI.LRL..BH
CI,LRL,,HH,ENZ,"[628306000.0, 629984000.0, 627467000.0]",35.47954,-117.68212,1340.0,-1.34,-16.165,-25.001,-1.34,SCEDC,CI.LRL..HH
CI,LRL,,HN,ENZ,"[213757.0, 213671.0, 213201.0]",35.47954,-117.68212,1340.0,-1.34,-16.165,-25.001,-1.34,SCEDC,CI.LRL..HN
CI,MPM,,BH,ENZ,"[627373000.0, 627373000.0, 627373000.0]",36.05799,-117.48901,1839.0,-1.839,1.351,39.167,-1.839,SCEDC,CI.MPM..BH
CI,MPM,,HH,ENZ,"[627369000.0, 627369000.0, 627369000.0]",36.05799,-117.48901,1839.0,-1.839,1.351,39.167,-1.839,SCEDC,CI.MPM..HH
CI,MPM,,HN,ENZ,"[213911.0, 214219.0, 213911.0]",36.05799,-117.48901,1839.0,-1.839,1.351,39.167,-1.839,SCEDC,CI.MPM..HN
CI,SLA,,BH,ENZ,"[622341000.0, 618995000.0, 616485000.0]",35.89095,-117.28332,1174.0,-1.174,19.925,20.655,-1.174,SCEDC,CI.SLA..BH
CI,SLA,,HH,ENZ,"[622336000.0, 618991000.0, 616481000.0]",35.89095,-117.28332,1174.0,-1.174,19.925,20.655,-1.174,SCEDC,CI.SLA..HH
CI,SLA,,HN,ENZ,"[214253.0, 213672.0, 213980.0]",35.89095,-117.28332,1174.0,-1.174,19.925,20.655,-1.174,SCEDC,CI.SLA..HN
CI,SRT,,BH,ENZ,"[633341000.0, 633341000.0, 633341000.0]",35.69235,-117.75051,667.0,-0.667,-22.312,-1.376,-0.667,SCEDC,CI.SRT..BH
CI,SRT,,HH,ENZ,"[629865000.0, 629865000.0, 629865000.0]",35.69235,-117.75051,667.0,-0.667,-22.312,-1.376,-0.667,SCEDC,CI.SRT..HH
CI,SRT,,HN,ENZ,"[214301.0, 213873.0, 214087.0]",35.69235,-117.75051,667.0,-0.667,-22.312,-1.376,-0.667,SCEDC,CI.SRT..HN
CI,TOW2,,BH,ENZ,"[626915000.0, 626915000.0, 626843000.0]",35.80856,-117.76488,685.0,-0.685,-23.579,11.522,-0.685,SCEDC,CI.TOW2..BH
CI,TOW2,,HH,ENZ,"[626911000.0, 626911000.0, 626839000.0]",35.80856,-117.76488,685.0,-0.685,-23.579,11.522,-0.685,SCEDC,CI.TOW2..HH
CI,TOW2,,HN,ENZ,"[213800.0, 214142.0, 214356.0]",35.80856,-117.76488,685.0,-0.685,-23.579,11.522,-0.685,SCEDC,CI.TOW2..HN
CI,WBM,,BH,ENZ,"[314573000.0, 314573000.0, 314573000.0]",35.60839,-117.89049,892.0,-0.892,-35.019,-10.65,-0.892,SCEDC,CI.WBM..BH
CI,WBM,,HH,ENZ,"[314573000.0, 314573000.0, 314573000.0]",35.60839,-117.89049,892.0,-0.892,-35.019,-10.65,-0.892,SCEDC,CI.WBM..HH
CI,WBM,,HN,ENZ,"[213550.0, 214064.0, 213550.0]",35.60839,-117.89049,892.0,-0.892,-35.019,-10.65,-0.892,SCEDC,CI.WBM..HN
CI,WCS2,,BH,ENZ,"[626910000.0, 626910000.0, 626838000.0]",36.02521,-117.76526,1143.0,-1.143,-23.549,35.561,-1.143,SCEDC,CI.WCS2..BH
CI,WCS2,,HH,ENZ,"[626910000.0, 626910000.0, 626838000.0]",36.02521,-117.76526,1143.0,-1.143,-23.549,35.561,-1.143,SCEDC,CI.WCS2..HH
CI,WCS2,,HN,ENZ,"[213757.0, 213329.0, 213415.0]",36.02521,-117.76526,1143.0,-1.143,-23.549,35.561,-1.143,SCEDC,CI.WCS2..HN
CI,WMF,,BH,ENZ,"[629963000.0, 631652000.0, 629963000.0]",36.11758,-117.85486,1537.4,-1.5373999999999999,-31.588,45.836,-1.537,SCEDC,CI.WMF..BH
CI,WMF,,HH,ENZ,"[626506000.0, 628185000.0, 626506000.0]",36.11758,-117.85486,1537.4,-1.5373999999999999,-31.588,45.836,-1.537,SCEDC,CI.WMF..HH
CI,WMF,,HN,ENZ,"[214087.0, 214087.0, 214087.0]",36.11758,-117.85486,1537.4,-1.5373999999999999,-31.588,45.836,-1.537,SCEDC,CI.WMF..HN
CI,WNM,,EH,Z,[69306200.0],35.8422,-117.90616,974.3,-0.9742999999999991,-36.332,15.298,-0.974,SCEDC,CI.WNM..EH
CI,WNM,,HN,ENZ,"[214054.0, 213926.0, 214054.0]",35.8422,-117.90616,974.3,-0.9742999999999991,-36.332,15.298,-0.974,SCEDC,CI.WNM..HN
CI,WRC2,,BH,ENZ,"[629145000.0, 629145000.0, 629145000.0]",35.9479,-117.65038,943.0,-0.9430000000000001,-13.207,26.961,-0.9430000000000001,SCEDC,CI.WRC2..BH
CI,WRC2,,HH,ENZ,"[629145000.0, 629145000.0, 629145000.0]",35.9479,-117.65038,943.0,-0.9430000000000001,-13.207,26.961,-0.9430000000000001,SCEDC,CI.WRC2..HH
CI,WRC2,,HN,ENZ,"[214227.0, 213970.0, 214056.0]",35.9479,-117.65038,943.0,-0.9430000000000001,-13.207,26.961,-0.9430000000000001,SCEDC,CI.WRC2..HN
CI,WRV2,,EH,Z,[71427500.0],36.00774,-117.8904,1070.0,-1.07,-34.836,33.66,-1.07,SCEDC,CI.WRV2..EH
CI,WRV2,,HN,ENZ,"[213883.0, 235224.0, 235139.0]",36.00774,-117.8904,1070.0,-1.07,-34.836,33.66,-1.07,SCEDC,CI.WRV2..HN
CI,WVP2,,EH,Z,[68019200.0],35.94939,-117.81769,1465.0,-1.465,-28.302,27.162,-1.465,SCEDC,CI.WVP2..EH
CI,WVP2,,HN,ENZ,"[213797.0, 213584.0, 213755.0]",35.94939,-117.81769,1465.0,-1.465,-28.302,27.162,-1.465,SCEDC,CI.WVP2..HN
"""
stations = pd.read_csv(StringIO(stations))

In [4]:
event = """time,x_km,y_km,z_km,magnitude,sigma_time,sigma_amp,cov_time_amp,event_index,gamma_score,longitude,latitude,depth_km
2019-07-04T21:00:56.902,-7.532,-2.654,14.816,2.161,0.323,0.399,0.013,33,83.998,-117.587,35.681,14.816"""
event = pd.read_csv(StringIO(event), parse_dates=['time'])

In [5]:
config = {
    "num_nodes": 1,
    "sampling_rate": 100,
    "degree2km": 111.1949,
    "channel": "HH*,BH*,EH*,HN*",
    "level": "response",
    "gamma": {
        "zmin_km": 0,
        "zmax_km": 60
    },
    "cctorch": {
        "sampling_rate": 100,
        "time_before": 0.25,
        "time_after": 1.0,
        "components": "ENZ123",
        "component_mapping": {
            "3": 0,
            "2": 1,
            "1": 2,
            "E": 0,
            "N": 1,
            "Z": 2
        }
    },
    "longitude0": -117.504,
    "latitude0": 35.705,
    "maxradius_degree": 0.5,
    "minlatitude": 35.205,
    "maxlatitude": 36.205,
    "minlongitude": -118.004,
    "maxlongitude": -117.004,
    "starttime": "2019-07-04T17:00:00",
    "endtime": "2019-07-05T00:00:00",
    "provider": [
        "SCEDC"
    ],
    "network": "CI"
}

In [6]:
proj =  Proj(f"+proj=sterea +lon_0={config['longitude0']} +lat_0={config['latitude0']} +units=km")
event['x_km'], event['y_km'] = proj(event['longitude'].values, event['latitude'].values)
stations['x_km'], stations['y_km'] = proj(stations['longitude'].values, stations['latitude'].values)

In [7]:
picks = picks.merge(stations, on='station_id', suffixes=('', '_station'))
picks = picks.merge(event, on='event_index', suffixes=('', '_event'))

In [8]:
picks.columns

Index(['station_id', 'phase_index', 'phase_time', 'phase_score', 'phase_type',
       'phase_polarity', 'phase_amplitude', 'id', 'timestamp', 'amp', 'type',
       'prob', 'event_index', 'gamma_score', 'network', 'station', 'location',
       'instrument', 'component', 'sensitivity', 'latitude', 'longitude',
       'elevation_m', 'depth_km', 'x_km', 'y_km', 'z_km', 'provider', 'time',
       'x_km_event', 'y_km_event', 'z_km_event', 'magnitude', 'sigma_time',
       'sigma_amp', 'cov_time_amp', 'gamma_score_event', 'longitude_event',
       'latitude_event', 'depth_km_event'],
      dtype='object')

In [9]:
# x0, y0 = proj(event['longitude'].values, event['latitude'].values)
x0, y0 = event['x_km'].values, event['y_km'].values

In [10]:
dr = 1 # km
nevent = 10
# theta = np.linspace(0, 2*np.pi, nevent+1)[:-1]
# x = x0 + dr*np.cos(theta)
# y = y0 + dr*np.sin(theta)
x = np.linspace(x0-dr, x0+dr, nevent)
# y = np.linspace(y0-dr, y0+dr, nevent)
y = np.ones(nevent)*y0
synthetic_events = []
synthetic_picks = []
vel = {"p": 6.0, "s": 6.0/1.73}

# synthetic_events.append(event)
# synthetic_picks.append(picks)

for i in range(nevent):

    tmp = event.copy()
    tmp['x_km'] = x[i]
    tmp['y_km'] = y[i]
    tmp['event_index'] = i
    lon, lat = proj(x[i], y[i], inverse=True)
    tmp['latitude'] = lat
    tmp['longitude'] = lon
    synthetic_events.append(tmp)

    tmp = picks.copy()
    x_sta = tmp['x_km'].values
    y_sta = tmp['y_km'].values
    z_sta = tmp['depth_km'].values
    v = tmp['phase_type'].str.lower().map(vel)
    tt0 = ((x_sta - x0)**2 + (y_sta - y0)**2 + z_sta**2)**0.5 / v
    tt = ((x_sta - x[i])**2 + (y_sta - y[i])**2 + z_sta**2)**0.5 / v

    tmp['phase_time'] = tmp['phase_time'] + pd.to_timedelta(tt - tt0, unit='s')
    tmp['event_index'] = i

    synthetic_picks.append(tmp)


In [11]:
synthetic_events = pd.concat(synthetic_events)
synthetic_picks = pd.concat(synthetic_picks)

In [12]:
synthetic_events.to_csv("local/demo/gamma/gamma_events.csv", index=False)
synthetic_picks.to_csv("local/demo/gamma/gamma_picks.csv", index=False)